### Deep Learning Classifier (Melinda Xiao-Devins)

Implement deep learning modle


In [1]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/anaconda/envs/em_hack_mac/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
#from tensorflow.python.layers.core import Dense
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))
# Check for a GPU
if not tf.test.gpu_device_name():
 warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
 print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.4.0


/anaconda/envs/em_hack_mac/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: No GPU found. Please use a GPU to train your neural network.
  # Remove the CWD from sys.path while we load stuff.


In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [4]:
# load dataset
#df = pd.read_csv('./data/ny_hmda_2015_normalize.csv', low_memory=False, header=0, delimiter=",")
df = pd.read_csv('./data/ny_hmda_2015_minmax.csv', low_memory=False, header=0, delimiter=",")

#print(dataframe.loc[:,:])
num_rows = df.shape[0]
num_col = df.shape[1]
print ("Total number of records: {}".format(num_rows))
print ("Toatl numver of features: {}".format(num_col))

dataset = df.values
# split into input (X) and output (Y) variables
X = dataset[:,1:num_col]
Y = dataset[:,0]

Total number of records: 376516
Toatl numver of features: 38


In [5]:
#Split into train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [6]:
#Create model

#create a sequencial model and add layers one at a time
model = Sequential()

# Create a fully connected network with 3 layers
#input layer, it has 12 neurons, it must have right number of inputs, which is the number of features
model.add(Dense(64, input_dim = num_col-1, activation = 'relu'))

# hideen layer has 8 neurons
model.add(Dense(32, activation = 'relu'))

# output layer has 1 neuron to predict
# Use sigmoid for output layer activation function to ensure network output is bw. 0 and 1
model.add(Dense(1, activation='sigmoid')) 

In [7]:
#Compile model

# loss function: logarithmic loss, which is binary_crossentropy for binary classification
# use 'adam' optimizer for gradient descent algorithm 
# collect accuracy during training
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Train the model
# epochs: a fixed number of iterations through the dataset
# batch size: the number of instances that are evaluated before a weight update in the network is performed 
model.fit(X_train, Y_train, epochs = 5, batch_size=500)

Epoch 1/5
301212/301212 [==============================] - 2s 5us/step - loss: 0.3690 - acc: 0.8086
Epoch 2/5
301212/301212 [==============================] - 1s 4us/step - loss: 0.3330 - acc: 0.8313
Epoch 3/5
301212/301212 [==============================] - 1s 4us/step - loss: 0.3301 - acc: 0.8327
Epoch 4/5
301212/301212 [==============================] - 1s 3us/step - loss: 0.3280 - acc: 0.8343
Epoch 5/5
301212/301212 [==============================] - 1s 3us/step - loss: 0.3275 - acc: 0.8344


In [9]:
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

75304/75304 [==============================] - 1s 15us/step

acc: 83.56%


In [10]:
# calculate predictions
predictions = model.predict(X)
print("the prediction bw. 0 and 1")
print(predictions)

# round predictions
rounded = [round(x[0]) for x in predictions]
print("the rounded prediction")
print(rounded[0:10])


the prediction bw. 0 and 1
[[ 0.6116094]
 [ 1.       ]
 [ 0.6312809]
 ..., 
 [ 1.       ]
 [ 1.       ]
 [ 1.       ]]
the rounded prediction
[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]


We are going to use scikit-learn to evaluate the model using stratified k-fold cross validation. This is a resampling technique that will provide an estimate of the performance of the model. It does this by splitting the data into k-parts, training the model on all parts except one which is held out as a test set to evaluate the performance of the model. This process is repeated k-times and the average score across all constructed models is used as a robust estimate of performance. It is stratified, meaning that it will look at the output values and attempt to balance the number of instances that belong to each class in the k-splits of the data.

To use Keras models with scikit-learn, we must use the KerasClassifier wrapper. This class takes a function that creates and returns our neural network model. It also takes arguments that it will pass along to the call to fit() such as the number of epochs and the batch size.

In [17]:
# larger model
def create_large_model():
    # Create a fully connected network with 5 layers
    model = Sequential()
    
    #input layer, it has 256 neurons, it must have right number of inputs, which is the number of features
    model.add(Dense(256, input_dim=num_col-1, kernel_initializer='normal', activation='relu'))
    
    # hideen layer has 128 neurons
    model.add(Dense(128, kernel_initializer='normal', activation='relu'))
        
    # hideen layer has 64 neurons
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
        
     # hideen layer has 32 neurons
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
  
    # output layer has 1 neuron to predict
    # Use sigmoid for output layer activation function to ensure network output is bw. 0 and 1
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

seed = 7
numpy.random.seed(seed)

# TODO: epochs = 150
large_model = KerasClassifier(build_fn=create_large_model, epochs=5, batch_size=10, verbose=1)

estimators = []
estimators.append(('standardize', StandardScaler()))

estimators.append(('mlp', large_model))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/5
338864/338864 [==============================] - 48s 142us/step - loss: 0.3324 - acc: 0.8337
Epoch 2/5
338864/338864 [==============================] - 49s 146us/step - loss: 0.3216 - acc: 0.8398
Epoch 3/5
338864/338864 [==============================] - 49s 146us/step - loss: 0.3196 - acc: 0.8412
Epoch 4/5
338864/338864 [==============================] - 49s 146us/step - loss: 0.3179 - acc: 0.8421
Epoch 5/5
37652/37652 [==============================] - 3s 70us/step
Epoch 1/5
338864/338864 [==============================] - 50s 146us/step - loss: 0.3322 - acc: 0.8335
Epoch 2/5
338864/338864 [==============================] - 48s 142us/step - loss: 0.3215 - acc: 0.8403
Epoch 3/5
338864/338864 [==============================] - 169s 499us/step - loss: 0.3190 - acc: 0.8415
Epoch 4/5
338864/338864 [==============================] - 461s 1ms/step - loss: 0.3179 - acc: 0.8425
Epoch 5/5
37652/37652 [==============================] - 3s 84us/step
Epoch 1/5
338864/338864 [===========

In [13]:
# calculate predictions

predictions = large_model.predict(X)
print("the prediction bw. 0 and 1")
print(predictions)

# round predictions
rounded = [round(x[0]) for x in predictions]
print("the rounded prediction")
print(rounded[0:10])


AttributeError: 'function' object has no attribute 'predict'

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(y_test, large_model.predict(x_test),average="macro")
print("precision={}, recall={}, fscore={}, support={}".format(precision, recall, fscore, support))
